<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [35]:
import pandas as pd
import csv as csv

In [36]:
# Load training data
train_df = pd.read_csv("data/train.csv", header=0)

In [37]:
train_df.head(3)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
train_label = train_df.values[:,0]
train_label

array([1, 0, 1, ..., 7, 6, 9])

In [39]:
from tensorflow.python.keras.utils import to_categorical
train_label = to_categorical(train_label)
train_label

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [40]:
train_data = train_df.values[:,1:]
train_data.reshape(42000,28,28)
train_data = train_data/255.

In [41]:
train_data = train_data.reshape(-1,28,28,1)
train_data.shape

(42000, 28, 28, 1)

In [42]:
# Load testning data
test_df = pd.read_csv("data/test.csv", header=0)
test_df.head(3)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
test_data = test_df.values
test_data.reshape(28000,28,28)
test_data = test_data/255.

In [44]:
test_data = test_data.reshape(-1,28,28,1)
test_data.shape

(28000, 28, 28, 1)

In [45]:
# とりあえず先頭の20000のみを使用
train_data = train_data[:20000]
train_label = train_label[:20000]

In [46]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense
model = Sequential()

In [47]:
def latest_model():
    model.add(
        Conv2D(
            filters=32,
            input_shape=(28, 28, 1),
            kernel_size=(3, 3),
            strides=(1, 1),
            padding='same',
            activation='relu'
        )
    )

    model.add(
        Conv2D(
            filters=32,
            kernel_size=(3, 3),
            strides=(1, 1),
            padding='same',
            activation='relu'
        )
    )
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Dropout(0.25))
    
    model.add(
        Conv2D(
            filters=64,
            kernel_size=(3, 3),
            strides=(1, 1),
            padding='same',
            activation='relu'
        )
    )
    
    model.add(
        Conv2D(
            filters=64,
            kernel_size=(3, 3),
            strides=(1, 1),
            padding='same',
            activation='relu'
        )
    )
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    
    model.add(Dense(units=512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=10, activation='softmax'))

In [48]:
latest_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 7, 7, 64)          0         
__________

In [49]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_model1 = model.fit(
    train_data,
    train_label,
    batch_size=32,
    epochs=3,
    validation_split=0.2
)

Train on 16000 samples, validate on 4000 samples
Epoch 1/3
16000/16000 [==============================] - 85s 5ms/step - loss: 0.3064 - acc: 0.9004 - val_loss: 0.0855 - val_acc: 0.9758
Epoch 2/3
16000/16000 [==============================] - 82s 5ms/step - loss: 0.1007 - acc: 0.9698 - val_loss: 0.0645 - val_acc: 0.9808
Epoch 3/3
16000/16000 [==============================] - 83s 5ms/step - loss: 0.0713 - acc: 0.9774 - val_loss: 0.0471 - val_acc: 0.9850


In [50]:
res = model.predict(test_data)

In [51]:
res[0]

array([4.0727340e-07, 1.8206194e-09, 9.9999928e-01, 4.6540059e-08,
       3.1770156e-11, 5.5491639e-11, 1.7752110e-10, 2.0542507e-07,
       7.4703614e-08, 8.8332752e-10], dtype=float32)

In [52]:
output = [0 for i in range(len(res))]
ids = [i for i in range(1,len(res)+1)]
for i in range(len(res)):
    output[i] = res[i].argmax()

In [53]:
submit_file = open("digit_recognizer_submit.csv", "w")
file_object = csv.writer(submit_file)
file_object.writerow(["ImageId", "Label"])
file_object.writerows(zip(ids, output))
submit_file.close()